# Examen 2 - Limpieza de Datos

## 1.- En un hospital se están haciendo pruebas de dos tratamientos experimentales a pacientes.

In [20]:
import numpy as np
import pandas as pd
nombre = ['Fernando', 'Alfonso', 'Santiago', 'Lili', 'Rosalia']
tratamiento_a = [25, 5, 6, 89, 10]
tratamiento_b = [0, 17, 15, 7, 9]

data_dict_1 = {
    'paciente':nombre,
    'tratamiento_a':tratamiento_a,
    'tratamiento_b': tratamiento_b
}

df_1 = pd.DataFrame.from_dict(data_dict_1)

df_1

,paciente,tratamiento_a,tratamiento_b
0,Fernando,25,0
1,Alfonso,5,17
2,Santiago,6,15
3,Lili,89,7
4,Rosalia,10,9


## 2.- Cada tratamiento incluye dos dosis.

In [21]:
nombre = ['tratamiento_a', 'tratamiento_b']
dosis_1 = [5, 9]
dosis_2 = [0, 7]

data_dict_2 = {
    'tratamiento':nombre,
    'dosis_1':dosis_1,
    'dosis_2': dosis_2
}

df_2 = pd.DataFrame.from_dict(data_dict_2)

df_2

,tratamiento,dosis_1,dosis_2
0,tratamiento_a,5,0
1,tratamiento_b,9,7


## 3.- Base de datos donde están registradas las dosis que recibió cada paciente.

In [22]:
df_pacientes= pd.melt(df_1, id_vars=['paciente'], value_vars=['tratamiento_a','tratamiento_b'])
df_pacientes.rename(columns={'variable':'tratamiento', 'value':'resultado'}, inplace=True)
df_pacientes

,paciente,tratamiento,resultado
0,Fernando,tratamiento_a,25
1,Alfonso,tratamiento_a,5
2,Santiago,tratamiento_a,6
3,Lili,tratamiento_a,89
4,Rosalia,tratamiento_a,10
5,Fernando,tratamiento_b,0
6,Alfonso,tratamiento_b,17
7,Santiago,tratamiento_b,15
8,Lili,tratamiento_b,7
9,Rosalia,tratamiento_b,9


## 4.- Comparación entre los tratamientos y la dosis recibida por paciente.

In [61]:
df_final = pd.merge(df_pacientes,df_2, how="left", on="tratamiento")
df_final

,paciente,tratamiento,resultado,dosis_1,dosis_2
0,Fernando,tratamiento_a,25,5,0
1,Alfonso,tratamiento_a,5,5,0
2,Santiago,tratamiento_a,6,5,0
3,Lili,tratamiento_a,89,5,0
4,Rosalia,tratamiento_a,10,5,0
5,Fernando,tratamiento_b,0,9,7
6,Alfonso,tratamiento_b,17,9,7
7,Santiago,tratamiento_b,15,9,7
8,Lili,tratamiento_b,7,9,7
9,Rosalia,tratamiento_b,9,9,7


La exportamos como csv

In [62]:
df_final.to_csv("df_final.csv")

## 5.- El hospital solicitó realizar algunas simulaciones de tratamientos, para esto se crea una base con datos ficticios.

In [23]:
np.random.seed(100)
n = 1000
id_paciente = np.random.randint(1000, size=n)
tratamiento_a = np.random.randint(low=10, high=100, size=n)
tratamiento_b = np.random.randint(low=15, high=90, size=n)


data_dict_pacientes = {
    'paciente':id_paciente,
    'tratamiento_a':tratamiento_a,
    'tratamiento_b': tratamiento_b
}

df_pacientes_2 = pd.DataFrame.from_dict(data_dict_pacientes)

df_pacientes_2['tratamiento_a'] = df_pacientes_2['tratamiento_a'].apply(
    lambda x: x if x>15 else np.nan)
df_pacientes_2['tratamiento_b'] = df_pacientes_2['tratamiento_b'].apply(
    lambda x: np.nan if x>80 else x)

df_pacientes_2.head()

,paciente,tratamiento_a,tratamiento_b
0,520,19.0,60.0
1,792,45.0,NaN
2,835,79.0,33.0
3,871,94.0,64.0
4,855,64.0,38.0


In [24]:
df_pacientes_2.to_csv("df-pacientes-2.csv")

## 6.- Busqueda de outliers

In [25]:
# Calcular cuartiles e IQR para tratamiento_a
Q1_a = df_pacientes_2['tratamiento_a'].quantile(0.25)
Q3_a = df_pacientes_2['tratamiento_a'].quantile(0.75)
IQR_a = Q3_a - Q1_a

# Definir límites para outliers en tratamiento_a
limite_inferior_a = Q1_a - 1.5 * IQR_a
limite_superior_a = Q3_a + 1.5 * IQR_a

# Identificar outliers en tratamiento_a
outliers_a = df_pacientes_2[(df_pacientes_2['tratamiento_a'] < limite_inferior_a) |
                            (df_pacientes_2['tratamiento_a'] > limite_superior_a)]

# Calcular cuartiles e IQR para tratamiento_b
Q1_b = df_pacientes_2['tratamiento_b'].quantile(0.25)
Q3_b = df_pacientes_2['tratamiento_b'].quantile(0.75)
IQR_b = Q3_b - Q1_b

# Definir límites para outliers en tratamiento_b
limite_inferior_b = Q1_b - 1.5 * IQR_b
limite_superior_b = Q3_b + 1.5 * IQR_b

# Identificar outliers en tratamiento_b
outliers_b = df_pacientes_2[(df_pacientes_2['tratamiento_b'] < limite_inferior_b) |
                            (df_pacientes_2['tratamiento_b'] > limite_superior_b)]

# Contar outliers
num_outliers_a = outliers_a.shape[0]
num_outliers_b = outliers_b.shape[0]

print(f"Número de outliers en tratamiento_a: {num_outliers_a}")
print(f"Número de outliers en tratamiento_b: {num_outliers_b}")

Número de outliers en tratamiento_a: 0
Número de outliers en tratamiento_b: 0


## 7.-Busqueda de datos perdidos

In [27]:
datos_perdidos = df_pacientes_2.isna().sum()

print("Datos perdidos por columna:")
print(datos_perdidos)

Datos perdidos por columna:
paciente           0
tratamiento_a     54
tratamiento_b    140
dtype: int64


## 8.- Imputación simple a la variable tratamiento_a.

In [51]:
# Calcular la media de la columna tratamiento_a (ignorando los NaN)
media_tratamiento_a = df_pacientes_2['tratamiento_a'].mean()

# Imputar los valores NaN con la media
df_pacientes_2['tratamiento_a'].fillna(media_tratamiento_a, inplace=True)

# Verificar que no hay datos perdidos después de la imputación
print("Datos perdidos en tratamiento_a después de la imputación:")
print(df_pacientes_2['tratamiento_a'].isna().sum())

Datos perdidos en tratamiento_a después de la imputación:
0


## 9.- Imputación múltiple a la variable tratamiento_b.

In [54]:
from sklearn.impute import IterativeImputer

# Crear el imputador con regresión bayesiana como estrategia
imputador = IterativeImputer(random_state=100)

# Seleccionar solo la columna a imputar y transformar a formato 2D
tratamiento_b_imputado = imputador.fit_transform(df_pacientes_2[['tratamiento_b']])

# Reemplazar los valores imputados en el DataFrame
df_pacientes_2['tratamiento_b'] = tratamiento_b_imputado

Exportamos la base final.

In [55]:
df_pacientes_2.to_csv("df-pacientes-2-imputado.csv")